In [1]:
import jax 
jax.config.update("jax_enable_x64", True)

In [2]:
import scvi
import scanpy as sc
import os
import pandas as pd
import numpy as np
from cfp.external import CFJaxSCVI

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 4
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/sciplex"
adata_train = sc.read(os.path.join(output_dir, f"adata_train_{split}.h5ad"))
adata_test = sc.read(os.path.join(output_dir, f"adata_test_{split}.h5ad"))
adata_ood = sc.read(os.path.join(output_dir, f"adata_ood_{split}.h5ad")) 




In [4]:
CFJaxSCVI.setup_anndata(adata_train)

n_latent=32
n_hidden=1024

vae = CFJaxSCVI(adata_train, gene_likelihood="normal", n_latent = n_latent, n_hidden = n_hidden)#, encoder_layers= layer, decoder_layers = layer)

vae.train(
    max_epochs=2000,
    batch_size=1024,
    plan_kwargs=dict(
        lr=1e-4,
    ),
    early_stopping=True,
    early_stopping_patience=20,
)

adata_ood.obsm["X_scVI"] = vae.get_latent_representation(adata_ood)
adata_ood.obsm["reconstruction"] = vae.get_reconstructed_expression(adata_ood, give_mean="True")
adata_ood.obsm["reconstruction2"] = vae.get_reconstructed_expression(adata_ood, give_mean="False")


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


INFO     Jax module moved to cuda:0.Note: Pytorch lightning will show GPU is not being used for the Trainer.       


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/icb/dominik.klein/mambaforge/envs/cfp/lib/pyth ...
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU availabl

Epoch 1/2000:   0%|          | 0/2000 [00:00<?, ?it/s]

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:385: You have overridden `transfer_batch_to_device` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.


Epoch 433/2000:  22%|██▏       | 433/2000 [31:16<1:53:11,  4.33s/it, v_num=1, train_loss_step=38.7, train_loss_epoch=37.2]
Monitored metric elbo_validation did not improve in the last 20 records. Best score: 37.162. Signaling Trainer to stop.
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [5]:
from sklearn.metrics import r2_score
r2_score(np.array(np.mean(adata_ood.X, axis=0))[0,:], np.mean(adata_ood.obsm["reconstruction"], axis=0))

0.9972022180207032

In [6]:
r2_score(np.array(np.mean(adata_ood.X, axis=0))[0,:], np.mean(adata_ood.obsm["reconstruction2"], axis=0))

0.9972022180207032

In [7]:
vae.save(f"/lustre/groups/ml01/workspace/ot_perturbation/models/vaes/sciplex/32_1024_{split}")